# Clustering Neighborhoods in Toronto 3

In [101]:
#from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ 
  - anaconda/linux-64::ca-certificates-2019.8.28-0, anaconda/linux-64::conda-4.8.1-py37_0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_2
  - anaconda/linux-64::ca-certificates-2019.8.28-0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_2, defaults/linux-64::conda-4.8.1-py37_0
  - anaconda/linux-64::conda-4.8.1-py37_0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_2, defaults/linux-64::ca-certificates-2019.8.28-0
  - anaconda/linux-64::openssl-1.1.1d-h7b6447c_2, defaults/linux-64::ca-certificates-2019.8.28-0, defaults/linux-64::conda-4.8.1-py37_0
  - anaconda/linux-64::conda-4.8.1-py37_0, defaults/linux-64::ca-certificates-2019.8.28-0, defaults/linux-64::openssl-1.1.1d-h7b6447c_2
  - defaults/linux-64::ca-certificates-2019.8.28-0, defaults/linux-64::conda-4.8.1-py37_0, defaults/linux-64::openssl-1.1.1d-h7b6447c_2
  - anaconda/linux-64::ca-certificates-2019.8.28-0, anaconda/linux-64::conda-4.8.1-py37_0, defaults/linux-64::openssl-1.1.1d-h7b6447c_2
  - anaconda/linux-64::c

Get Neighborhood data from wikipedia, extract the required section and display the columns.

In [67]:
wiki_df= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wiki_df
df_sub=wiki_df[0]
df_sub.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

Create a new dataframe with Boroughs that dont contain the value "Not assigned". Then display the unique Boroughs to confirm.

In [68]:
df_sub1=df_sub[df_sub['Borough']!="Not assigned"]
df_sub1['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Scarborough',
       'East York', 'Etobicoke', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Display the dataframe

In [69]:
df_sub1

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


Restore default index

In [70]:
df_sub1.reset_index(inplace=True)
df_sub1

,index,Postcode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor
...,...,...,...,...
205,281,M8Z,Etobicoke,Kingsway Park South West
206,282,M8Z,Etobicoke,Mimico NW
207,283,M8Z,Etobicoke,The Queensway West
208,284,M8Z,Etobicoke,Royal York South West


Check if there are any empty values in the dataframe

In [7]:
df_sub1.isnull().sum()

index           0
Postcode        0
Borough         0
Neighborhood    0
dtype: int64

In [8]:
df_sub1['Postcode'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

Remove duplicate entries of Postcode by grouping and joining the Neigborhood values into one row,, separated by commas.

In [71]:
#dup=pd.DataFrame(df_sub1.groupby(['Postcode','Borough'])['Neighborhood'].agg(sum() if x.dtype=='float64' else ' '.join(x)
dup=df_sub1.groupby(['Postcode','Borough'],as_index=False)['Neighborhood'].agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))    
dup

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Display which neighborhood  has "Not assigned"

In [72]:
dup[dup['Neighborhood']=="Not assigned"]

,Postcode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


Run through dataframe looking for neighborhoods with not assigned and replace with Borough value in such case.

In [75]:
for a in range(0,len(dup)-1):
    for i in dup['Neighborhood']:
        if i=="Not assigned":
            dup.iloc[a,2]=dup.iloc[a,1]     

Verify that not assinged was changed to borough value

In [76]:
dup[dup['Neighborhood']=="Queen's Park"]

,Postcode,Borough,Neighborhood
93,M9A,Queen's Park,Queen's Park


Display the sape of the dataframe.

In [77]:
dup.shape

(103, 3)

In [81]:
!pip install geocoder

     |████████████████████████████████| 102kB 382kB/s ta 0:00:01


In [83]:
import geocoder
import folium

In [86]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='

KeyboardInterrupt: 

In [94]:
df_geo=pd.read_csv("https://cocl.us/Geospatial_data")
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [98]:
dup1=dup.rename(columns={'Postcode':'Postal Code'})

In [99]:
df_df=dup1.merge(df_geo,on='Postal Code')

In [100]:
df_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Scarborough,43.806686,-79.194353
1,M1C,Scarborough,Scarborough,43.784535,-79.160497
2,M1E,Scarborough,Scarborough,43.763573,-79.188711
3,M1G,Scarborough,Scarborough,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [102]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [103]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_df['Latitude'], df_df['Longitude'], df_df['Borough'], df_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto